In [ ]:
import cv2
import numpy as np
import random

# Video parameters
width = 640
height = 480
fps = 30
duration_seconds = 10
video_filename = 'random_video.mp4'

# Calculate the number of frames
num_frames = fps * duration_seconds

# Create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 format
video_writer = cv2.VideoWriter(video_filename, fourcc, fps, (width, height))

# Generate and write random frames
for _ in range(num_frames):
    # Create a random frame with random colors
    frame = np.random.randint(0, 256, (height, width, 3), dtype=np.uint8)
    
    # Write the frame to the video
    video_writer.write(frame)

# Release the video writer
video_writer.release()

print(f"Random video generated as {video_filename}")


In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == "__main__":
    app.run()

In [ ]:
from flask import Flask, send_from_directory, render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video')
def video():
    return send_from_directory('.', 'random_video.mp4')

if __name__ == '__main__':
    app.run(debug=False)


In [ ]:
import mimetypes

In [ ]:
import socket
import mimetypes
import os

def serve_file(filename, client_socket):
    try:
        with open(filename, 'rb') as f:
            content = f.read()
            mime_type, _ = mimetypes.guess_type(filename)
            if mime_type is None:
                mime_type = 'application/octet-stream'
            response = f"HTTP/1.1 200 OK\r\nContent-Type: {mime_type}\r\nContent-Length: {len(content)}\r\n\r\n"
            client_socket.sendall(response.encode() + content)
    except FileNotFoundError:
        response = "HTTP/1.1 404 Not Found\r\n\r\n"
        client_socket.sendall(response.encode())

def handle_request(client_socket):
    request = client_socket.recv(1024).decode()
    request_line = request.splitlines()[0]
    requested_file = request_line.split(' ')[1]
    if requested_file == '/':
        requested_file = '/index.html'
    else:
        requested_file = requested_file[1:]
    if not os.path.isfile(requested_file):
        requested_file = '404.html'
    serve_file(requested_file, client_socket)
    client_socket.close()

def run_server(host='127.0.0.1', port=8082):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(5)
    print(f"Serving on http://{host}:{port}")

    while True:
        client_socket, _ = server_socket.accept()
        handle_request(client_socket)

if __name__ == '__main__':
    run_server()


In [ ]:
import socket
import mimetypes
import os

def log_connection(address):
    print(f"Client connected from {address}")

def serve_file(filename, client_socket):
    try:
        with open(filename, 'rb') as f:
            content = f.read()
            mime_type, _ = mimetypes.guess_type(filename)
            if mime_type is None:
                mime_type = 'application/octet-stream'
            response = f"HTTP/1.1 200 OK\r\nContent-Type: {mime_type}\r\nContent-Length: {len(content)}\r\n\r\n"
            client_socket.sendall(response.encode() + content)
    except FileNotFoundError:
        response = "HTTP/1.1 404 Not Found\r\n\r\n"
        client_socket.sendall(response.encode())

def handle_ping(client_socket):
    response = "HTTP/1.1 200 OK\r\nContent-Type: text/plain\r\n\r\nPong!"
    client_socket.sendall(response.encode())

def handle_request(client_socket, address):
    request = client_socket.recv(1024).decode()
    request_line = request.splitlines()[0]
    requested_file = request_line.split(' ')[1]
    
    if requested_file == '/':
        requested_file = '/index.html'
    elif requested_file == '/ping':
        handle_ping(client_socket)
        client_socket.close()
        return
    else:
        requested_file = requested_file[1:]
    
    if not os.path.isfile(requested_file):
        requested_file = '404.html'
    
    serve_file(requested_file, client_socket)
    client_socket.close()

def run_server(host='localhost', port=8080):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(5)
    print(f"Serving on http://{host}:{port}")

    while True:
        client_socket, address = server_socket.accept()
        log_connection(address)
        handle_request(client_socket, address)

if __name__ == '__main__':
    run_server()
